<a href="https://colab.research.google.com/github/gagatadi/machine-learning/blob/main/Week%20%237%20Distributed%20XGBoost%20with%20Ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ray


     |████████████████████████████████| 54.7 MB 35 kB/s 
     |████████████████████████████████| 72 kB 623 kB/s 


In [2]:
pip install xgboost_ray

     |████████████████████████████████| 136 kB 9.6 MB/s 


In [3]:
pip install sklearn

In [4]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

train_x, train_y = load_breast_cancer(return_X_y=True)
train_set = RayDMatrix(train_x, train_y)

evals_result = {}
bst = train(
    {
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    train_set,
    evals_result=evals_result,
    evals=[(train_set, "train")],
    verbose_eval=False,
    ray_params=RayParams(num_actors=2, cpus_per_actor=1))

bst.save_model("model.xgb")
print("Final training error: {:.4f}".format(
    evals_result["train"]["error"][-1]))

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 11:09:25,112	INFO main.py:971 -- [RayXGBoost] Created 2 new actors (2 total actors). Waiting until actors are ready for training.
2021-11-12 11:09:27,253	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.
(pid=301) [11:09:27] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pid=301) [11:09:27] Tree method is automatically selected to be 'approx' for distributed training.
(pid=325) [11:09:27] WARNING: /workspace/src/learner.cc:622: Tree method is automatically selected to be 'approx' for distributed training.
(pi

Final training error: 0.0053


In [5]:
from xgboost_ray import RayDMatrix, RayParams, predict
from sklearn.datasets import load_breast_cancer
import xgboost as xgb

data, labels = load_breast_cancer(return_X_y=True)

dpred = RayDMatrix(data, labels)

bst = xgb.Booster(model_file="model.xgb")
pred_ray = predict(bst, dpred, ray_params=RayParams(num_actors=2))

print(pred_ray)

/usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
  warnings.warn(LEGACY_WARNING)
2021-11-12 11:09:44,423	INFO main.py:1535 -- [RayXGBoost] Created 2 remote actors.
2021-11-12 11:09:46,565	INFO main.py:1552 -- [RayXGBoost] Starting XGBoost prediction.


[0.07511145 0.04715708 0.02642454 0.11464816 0.08779665 0.19609766
 0.02642454 0.03200296 0.03968306 0.06930447 0.09290799 0.02642454
 0.03476055 0.05490047 0.05012454 0.02642454 0.03337562 0.02642454
 0.02642454 0.9528658  0.9751126  0.9751126  0.07511145 0.02642454
 0.02642454 0.04102892 0.03118352 0.02642454 0.02642454 0.08779665
 0.02642454 0.03476055 0.02642454 0.02642454 0.02642454 0.02642454
 0.0665335  0.9690314  0.33619487 0.0696831  0.5729184  0.15662025
 0.03451282 0.03200296 0.12301622 0.04102892 0.9751126  0.04096693
 0.97315884 0.9234187  0.9751126  0.9751126  0.9751126  0.03059936
 0.09855346 0.9751126  0.02642454 0.03755146 0.9751126  0.97315884
 0.9751126  0.97315884 0.02642454 0.9751126  0.03120249 0.03120249
 0.97315884 0.9751126  0.8630876  0.9751126  0.02642454 0.9751126
 0.02642454 0.18486539 0.9751126  0.02642454 0.9684864  0.07511145
 0.04528174 0.9751126  0.97315884 0.8423582  0.02642454 0.02642454
 0.9751126  0.02642454 0.18525673 0.02642454 0.9610139  0.91992

In [6]:
from xgboost_ray import RayDMatrix, RayParams, train
from sklearn.datasets import load_breast_cancer

num_actors = 1
num_cpus_per_actor = 1

ray_params = RayParams(
    num_actors=num_actors, cpus_per_actor=num_cpus_per_actor)

def train_model(config):
    train_x, train_y = load_breast_cancer(return_X_y=True)
    train_set = RayDMatrix(train_x, train_y)

    evals_result = {}
    bst = train(
        params=config,
        dtrain=train_set,
        evals_result=evals_result,
        evals=[(train_set, "train")],
        verbose_eval=False,
        ray_params=ray_params)
    bst.save_model("model.xgb")

from ray import tune

# Specify the hyperparameter search space.
config = {
    "tree_method": "approx",
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "error"],
    "eta": tune.loguniform(1e-4, 1e-1),
    "subsample": tune.uniform(0.5, 1.0),
    "max_depth": tune.randint(1, 9)
}

# Make sure to use the `get_tune_resources` method to set the `resources_per_trial`
analysis = tune.run(
    train_model,
    config=config,
    metric="train-error",
    mode="min",
    num_samples=4,
    resources_per_trial=ray_params.get_tune_resources())
print("Best hyperparameters", analysis.best_config)

2021-11-12 11:09:56,712	WARNING function_runner.py:564 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-11-12 11:09:56,730	INFO logger.py:606 -- pip install 'ray[tune]' to see TensorBoard files.
2021-11-12 11:09:56,735	WARNING callback.py:115 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Trial name,status,loc,eta,max_depth,subsample
train_model_11b5b_00000,PENDING,,0.000159847,1,0.795395
train_model_11b5b_00001,PENDING,,0.0198814,4,0.823168
train_model_11b5b_00002,PENDING,,0.00474631,1,0.89023
train_model_11b5b_00003,PENDING,,0.00956784,6,0.967491


(pid=405) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:166: UserWarning: You are using `xgboost_ray` with a legacy XGBoost version (version 0.90). While we try to support older XGBoost versions, please note that this library is only fully tested and supported for XGBoost >= 1.4. Please consider upgrading your XGBoost version (`pip install -U xgboost`).
(pid=405)   warnings.warn(LEGACY_WARNING)
(pid=405) /usr/local/lib/python3.7/dist-packages/xgboost_ray/main.py:422: UserWarning: `num_actors` in `ray_params` is smaller than 2 (1). XGBoost will NOT be distributed!
(pid=405)   f"`num_actors` in `ray_params` is smaller than 2 "
(pid=405) 2021-11-12 11:09:59,041	INFO main.py:971 -- [RayXGBoost] Created 1 new actors (1 total actors). Waiting until actors are ready for training.
(pid=405) 2021-11-12 11:10:01,655	INFO main.py:1016 -- [RayXGBoost] Starting XGBoost training.


Trial name,status,loc,eta,max_depth,subsample
train_model_11b5b_00000,RUNNING,172.28.0.2:405,0.000159847,1,0.795395
train_model_11b5b_00001,PENDING,,0.0198814,4,0.823168
train_model_11b5b_00002,PENDING,,0.00474631,1,0.89023
train_model_11b5b_00003,PENDING,,0.00956784,6,0.967491


Result for train_model_11b5b_00000:
  date: 2021-11-12_11-10-02
  done: false
  experiment_id: a9b8fbb4302d430c8fa7ca1a4dc989ec
  hostname: b1df80ce45c6
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 405
  time_since_restore: 3.7044436931610107
  time_this_iter_s: 3.7044436931610107
  time_total_s: 3.7044436931610107
  timestamp: 1636715402
  timesteps_since_restore: 0
  train-error: 0.084359
  train-logloss: 0.693036
  training_iteration: 1
  trial_id: 11b5b_00000
  
Result for train_model_11b5b_00000:
  date: 2021-11-12_11-10-02
  done: true
  experiment_id: a9b8fbb4302d430c8fa7ca1a4dc989ec
  experiment_tag: 0_eta=0.00015985,max_depth=1,subsample=0.7954
  hostname: b1df80ce45c6
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 405
  time_since_restore: 3.7710139751434326
  time_this_iter_s: 0.0057337284088134766
  time_total_s: 3.7710139751434326
  timestamp: 1636715402
  timesteps_since_restore: 0
  train-error: 0.075571
  train-logloss: 0.692036
  training_ite

...

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_11b5b_00003,RUNNING,172.28.0.2:717,0.00956784,6,0.967491,,,,
train_model_11b5b_00000,TERMINATED,172.28.0.2:405,0.000159847,1,0.795395,10,3.77101,0.692036,0.075571
train_model_11b5b_00001,TERMINATED,172.28.0.2:509,0.0198814,4,0.823168,10,3.75604,0.545929,0.008787
train_model_11b5b_00002,TERMINATED,172.28.0.2:613,0.00474631,1,0.89023,10,3.76355,0.661302,0.063269


Result for train_model_11b5b_00003:
  date: 2021-11-12_11-10-19
  done: false
  experiment_id: 79282dc2b8814320a218d0b83a50b524
  hostname: b1df80ce45c6
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  pid: 717
  time_since_restore: 3.7023210525512695
  time_this_iter_s: 3.7023210525512695
  time_total_s: 3.7023210525512695
  timestamp: 1636715419
  timesteps_since_restore: 0
  train-error: 0.02109
  train-logloss: 0.684639
  training_iteration: 1
  trial_id: 11b5b_00003
  
Result for train_model_11b5b_00003:
  date: 2021-11-12_11-10-19
  done: true
  experiment_id: 79282dc2b8814320a218d0b83a50b524
  experiment_tag: 3_eta=0.0095678,max_depth=6,subsample=0.96749
  hostname: b1df80ce45c6
  iterations_since_restore: 10
  node_ip: 172.28.0.2
  pid: 717
  time_since_restore: 3.771862030029297
  time_this_iter_s: 0.005606889724731445
  time_total_s: 3.771862030029297
  timestamp: 1636715419
  timesteps_since_restore: 0
  train-error: 0.008787
  train-logloss: 0.614291
  training_iterati

Trial name,status,loc,eta,max_depth,subsample,iter,total time (s),train-logloss,train-error
train_model_11b5b_00000,TERMINATED,172.28.0.2:405,0.000159847,1,0.795395,10,3.77101,0.692036,0.075571
train_model_11b5b_00001,TERMINATED,172.28.0.2:509,0.0198814,4,0.823168,10,3.75604,0.545929,0.008787
train_model_11b5b_00002,TERMINATED,172.28.0.2:613,0.00474631,1,0.89023,10,3.76355,0.661302,0.063269
train_model_11b5b_00003,TERMINATED,172.28.0.2:717,0.00956784,6,0.967491,10,3.77186,0.614291,0.008787


(pid=717) 2021-11-12 11:10:19,958	INFO main.py:1498 -- [RayXGBoost] Finished XGBoost training on training data with total N=569 in 3.79 seconds (1.12 pure XGBoost training time).
2021-11-12 11:10:20,115	INFO tune.py:630 -- Total run time: 23.40 seconds (23.23 seconds for the tuning loop).


Best hyperparameters {'tree_method': 'approx', 'objective': 'binary:logistic', 'eval_metric': ['logloss', 'error'], 'eta': 0.019881359120215953, 'subsample': 0.8231678804096689, 'max_depth': 4}
